# Introduction to Logistic Regression Using TF 2.0

**Learning Objectives**


1. Build a neural network that classifies images.
2. Train this neural network.
3. Evaluate the accuracy of the model.


## Introduction 

This short introduction uses [Keras](https://keras.io/), a high-level API to build and train models in TensoFlow.  In this lab, you Load and prepare the MNIST dataset, convert the samples from integers to floating-point numbers, build and train a neural network that classifies images and the evaluate then accuracy of the model.

Each learning objective will correspond to a __#TODO__ in the [student lab notebook](https://github.com/GoogleCloudPlatform/training-data-analyst/blob/master/courses/machine_learning/deepdive2/introduction_to_tensorflow/labs/intro_logistic_regression_TF2.0.ipynb) -- try to complete that notebook first before reviewing this solution notebook.

## Load necessary libraries 
We will start by importing the necessary libraries for this lab.

In [20]:
import tensorflow as tf
print("TensorFlow version: ",tf.version.VERSION)

TensorFlow version:  2.1.0


Load and prepare the [MNIST dataset](http://yann.lecun.com/exdb/mnist/). Convert the samples from integers to floating-point numbers:

In [21]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

**Lab Task 1:** Build the `tf.keras.Sequential` model by stacking layers. Choose an optimizer and loss function for training:

In [22]:
model = # TODO 1 -- Your code here.

For each example the model returns a vector of "[logits](https://developers.google.com/machine-learning/glossary#logits)" or "[log-odds](https://developers.google.com/machine-learning/glossary#log-odds)" scores, one for each class.

In [23]:
predictions = model(x_train[:1]).numpy()
predictions


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



array([[ 0.06166657,  0.07144614, -0.07372011,  0.3451226 , -0.06205732,
        -0.23894641, -0.00426888,  0.38629198,  0.11753443,  0.21888584]],
      dtype=float32)

The `tf.nn.softmax` function converts these logits to "probabilities" for each class: 

In [24]:
tf.nn.softmax(predictions).numpy()

array([[0.09631761, 0.09726418, 0.0841217 , 0.1278819 , 0.08510853,
        0.07131012, 0.09017171, 0.1332566 , 0.10185182, 0.11271589]],
      dtype=float32)

Note: It is possible to bake this `tf.nn.softmax` in as the activation function for the last layer of the network. While this can make the model output more directly interpretable, this approach is discouraged as it's impossible to
provide an exact and numerically stable loss calculation for all models when using a softmax output. 

The `losses.SparseCategoricalCrossentropy` loss takes a vector of logits and a `True` index and returns a scalar loss for each example.

**Lab Task #2:** Usage of losses.SparseCategoricalCrossentropy with logits vectors and a True index.

In [25]:
loss_fn = # TODO 2 -- Your code here.

This loss is equal to the negative log probability of the true class:
It is zero if the model is sure of the correct class.

This untrained model gives probabilities close to random (1/10 for each class), so the initial loss should be close to `-tf.log(1/10) ~= 2.3`.

In [26]:
loss_fn(y_train[:1], predictions).numpy()

2.6407173

In [27]:
model.compile(optimizer='adam',
              loss=loss_fn,
              metrics=['accuracy'])

The `Model.fit` method adjusts the model parameters to minimize the loss: 

In [28]:
model.fit(x_train, y_train, epochs=5)

Train on 60000 samples
Epoch 1/5
60000/60000 [==============================] - 4s 74us/sample - loss: 0.2948 - accuracy: 0.9159
Epoch 2/5
60000/60000 [==============================] - 4s 68us/sample - loss: 0.1449 - accuracy: 0.9575
Epoch 3/5
60000/60000 [==============================] - 4s 67us/sample - loss: 0.1086 - accuracy: 0.9669
Epoch 4/5
60000/60000 [==============================] - 4s 67us/sample - loss: 0.0890 - accuracy: 0.9722
Epoch 5/5
60000/60000 [==============================] - 4s 67us/sample - loss: 0.0760 - accuracy: 0.9761


The `Model.evaluate` method checks the models performance, usually on a "[Validation-set](https://developers.google.com/machine-learning/glossary#validation-set)" or "[Test-set](https://developers.google.com/machine-learning/glossary#test-set)".

In [10]:
model.evaluate(x_test,  y_test, verbose=2)

10000/10000 - 0s - loss: 0.0789 - accuracy: 0.9762


[0.07894639570089057, 0.9762]

The image classifier is now trained to ~98% accuracy on this dataset. To learn more, read the [TensorFlow tutorials](https://www.tensorflow.org/tutorials/).

If you want your model to return a probability, you can wrap the trained model, and attach the softmax to it:

In [11]:
probability_model = tf.keras.Sequential([
  model,
  tf.keras.layers.Softmax()
])

In [12]:
probability_model(x_test[:5])

<tf.Tensor: shape=(5, 10), dtype=float32, numpy=
array([[4.16968859e-08, 6.23608756e-08, 5.28075470e-05, 2.82978057e-03,
        1.16726082e-11, 1.71175685e-07, 3.04637446e-13, 9.97084558e-01,
        4.06323215e-06, 2.85600290e-05],
       [1.44917021e-08, 2.85007991e-05, 9.99968171e-01, 2.84016392e-06,
        5.33308461e-15, 2.64569479e-07, 3.80620824e-09, 1.71865629e-11,
        2.15737089e-07, 1.31360444e-14],
       [4.27655920e-08, 9.99692202e-01, 3.80819870e-06, 4.24063728e-06,
        1.78703067e-05, 1.56593114e-06, 4.08388269e-06, 2.54444923e-04,
        2.10298167e-05, 6.26707333e-07],
       [9.99903679e-01, 5.74413441e-08, 6.42219311e-05, 2.98295333e-07,
        6.06606605e-08, 2.66647930e-05, 7.69448434e-07, 3.18793695e-06,
        2.18482214e-07, 8.51092807e-07],
       [5.69916847e-06, 1.11318251e-07, 1.25890219e-04, 7.61389970e-07,
        9.85019803e-01, 1.97083864e-06, 1.77188417e-06, 1.21961719e-04,
        3.18770253e-05, 1.46901319e-02]], dtype=float32)>